In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
%cd /content/drive/My\ Drive/Pattern\ Lab/Datasets/Datasets\ SL/classified_datasets
path = "/content/drive/My\ Drive/Pattern\ Lab/Datasets/Datasets\ SL/classified_datasets"

/content/drive/My Drive/Pattern Lab/Datasets/Datasets SL/classified_datasets


In [3]:
# all library imports here
import pandas as pd
import numpy as np
import os
import pickle

In [4]:
a_file = open("index_dictionary.pkl", "rb")
index_dictionary = pickle.load(a_file)
print(index_dictionary)

{'binary': [0, 163], 'CKSNAP1': [164, 195], 'CKSNAP3': [196, 259], 'CKSNAP5': [260, 355], 'CKSNAP7': [356, 483], 'CKSNAP9': [484, 643], 'ENAC5': [644, 791], 'ENAC10': [792, 939], 'DAC': [940, 981], '_EIIP': [982, 1022], 'PseEIIP': [1023, 1086], 'NCP': [1087, 1209], 'Kmer1': [1210, 1213], 'Kmer2': [1214, 1229], 'Kmer3': [1230, 1293], 'Kmer4': [1294, 1549], 'Kmer5': [1550, 2573], 'TAC': [2574, 2587]}


In [5]:
import random

def getList(dict): 
    return list(dict.keys()) 

       
keyList = getList(index_dictionary)
print(type(keyList))

random_feature = [] 

for i in range(4):
  random_feature.append(random.sample(keyList, 5))

print(random_feature )

<class 'list'>
[['CKSNAP9', 'CKSNAP3', 'CKSNAP1', 'Kmer3', 'CKSNAP5'], ['PseEIIP', 'Kmer3', 'Kmer2', 'Kmer4', 'Kmer5'], ['CKSNAP1', 'CKSNAP3', 'DAC', 'CKSNAP7', 'Kmer5'], ['Kmer4', '_EIIP', 'DAC', 'Kmer3', 'PseEIIP']]


In [6]:
Xa_train, Xa_test, ya_train, ya_test = np.load("xa_train.npy"), np.load("xa_test.npy"), np.load("ya_train.npy"), np.load("ya_test.npy")
Xc_train, Xc_test, yc_train, yc_test = np.load("xc_train.npy"), np.load("xc_test.npy"), np.load("yc_train.npy"), np.load("yc_test.npy")
Xd_train, Xd_test, yd_train, yd_test = np.load("xd_train.npy"), np.load("xd_test.npy"), np.load("yd_train.npy"), np.load("yd_test.npy")
print(Xa_train.shape, Xa_test.shape, ya_train.shape, ya_test.shape)
print(Xc_train.shape, Xc_test.shape, yc_train.shape, yc_test.shape)
print(Xd_train.shape, Xd_test.shape, yd_train.shape, yd_test.shape)

(3690, 2568) (262, 2568) (3690, 1) (262, 1)
(2898, 2568) (206, 2568) (2898, 1) (206, 1)
(3300, 2568) (234, 2568) (3300, 1) (234, 1)


In [7]:
#pa_test = np.concatenate((pa_test, dt), axis=1)
xa_trains = []
xa_tests = []
for i, feature_groups in enumerate(random_feature):
  #no_of_features = 0
  #for j in feature_groups:
   # no_of_features = no_of_features + index_dictionary[j][1]-index_dictionary[j][0]+1
  #print(no_of_features)
  xa_train_temp = np.concatenate([Xa_train[ : ,  index_dictionary[x][0] : index_dictionary[x][1]+1] for x in feature_groups], axis = 1)
  xa_trains.append(xa_train_temp)
  xa_test_temp = np.concatenate([Xa_test[ : ,  index_dictionary[x][0] : index_dictionary[x][1]+1] for x in feature_groups], axis = 1)
  xa_tests.append(xa_test_temp)
  print(xa_trains[i].shape)
  print(xa_tests[i].shape)

(3690, 416)
(262, 416)
(3690, 1418)
(262, 1418)
(3690, 1284)
(262, 1284)
(3690, 467)
(262, 467)


In [8]:
from sklearn.model_selection import train_test_split
from sklearn import svm
import random

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.metrics import confusion_matrix, matthews_corrcoef
from xgboost import XGBClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import StratifiedKFold
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

from tabulate import tabulate
from IPython.display import clear_output

In [9]:
def avg(avg_list):
  return sum(avg_list)/len(avg_list)

In [10]:
cv = StratifiedKFold( n_splits=5, shuffle=True) # 5 fold cross validation

# Define all classifier
svc = SVC() # Support vector classifier(SVC)
logisticRegression = LogisticRegression(max_iter=500)
xgbC = XGBClassifier()
adaBC = AdaBoostClassifier()
rfC = RandomForestClassifier()
classifiers = [svc, logisticRegression, xgbC, adaBC, rfC]
classifier_name = ["SVC", "LogisticRegression", "XGB",  "AdaBoost", "RandomForestClassifier"]

rows = []
headers = ["Classifier", "Random-Set", "Train-Accuracy", "Test-Accuracy", "Ind-Accuracy","Train-Specificity", "Test-Specificity", "Ind-Specificy", "Train-Sensitivity", "Test-Sensitivity", "Ind-Sensitivity", "Train-MCC", "Test-MCC", "Ind-MCC"]
for n, clf in enumerate(classifiers):
  temp_row = []
  for i, x in enumerate(xa_trains):
    train_accuracy=[]
    train_specificity = []
    train_sensitivity = []
    train_mcc = []
    test_accuracy=[]
    test_specificity = []
    test_sensitivity = []
    test_mcc = []
    ind_accuracy=[]
    ind_specificity = []
    ind_sensitivity = []
    ind_mcc = []
    for train_index, test_index in cv.split(x, ya_train):
        x_train, x_test = x[train_index], x[test_index]
        y_train, y_test = ya_train[train_index], ya_train[test_index]
        clf.fit(x_train, y_train.ravel())
        test_pred = clf.predict(x_test) # prediction of train dataset cross validation
        train_pred = clf.predict(x_train) # prediction of test dataset cross validation
        ind_pred = clf.predict(xa_tests[i]) # prediction of independent test dataset

        ## Accuracy
        train_accuracy.append(accuracy_score(y_train, train_pred))
        test_accuracy.append(accuracy_score(y_test, test_pred))
        ind_accuracy.append(accuracy_score(ya_test, ind_pred))

        ## confusion metrics
        tn, fp, fn, tp = confusion_matrix(y_train, train_pred).ravel()
        train_specificity.append(tn / (tn + fp)) # Specificity
        train_sensitivity.append(tp / (tp + fn)) # Sensitivity

        tn, fp, fn, tp = confusion_matrix(y_test, test_pred).ravel()
        test_specificity.append(tn / (tn + fp))
        test_sensitivity.append(tp / (tp + fn))

        tn, fp, fn, tp = confusion_matrix(ya_test, ind_pred).ravel()
        ind_specificity.append(tn / (tn + fp))
        ind_sensitivity.append(tp / (tp + fn))

        ## MCC
        train_mcc.append(matthews_corrcoef(y_train, train_pred))
        test_mcc.append(matthews_corrcoef(y_test, test_pred))
        ind_mcc.append(matthews_corrcoef(ya_test, ind_pred))

    #print("********\nRandom set -", i+1)
    #print("Train-Accuracy = ", avg(train_accuracy))
    #print("Test-Accuracy = ",avg(test_accuracy))
    #print("IND-Accuracy = ",avg(ind_accuracy))
    #print("Train-Specificity = ", avg(train_specificity))
    #print("Test-Specificity = ", avg(test_specificity))
    #print("IND-Specificity = ", avg(ind_specificity))
    #print("Train-Sensitivity = ", avg(train_sensitivity))
    #print("Test-Sensitivity = ", avg(test_sensitivity))
    #print("IND-Sensitivity = ", avg(ind_sensitivity))
    #print("Train-MCC = ", avg(train_mcc))
    #print("Test-MCC = ", avg(test_mcc))
    #print("IND-MCC = ", avg(ind_mcc))
    #print("\n\n")
    temp_row.append([classifier_name[n] if i==0 else " ", "Set0"+str(i+1), avg(train_accuracy), avg(test_accuracy), avg(ind_accuracy), avg(train_specificity), avg(test_specificity), avg(ind_specificity), 
                 avg(train_sensitivity), avg(test_sensitivity),  avg(ind_sensitivity), avg(train_mcc), avg(test_mcc),  avg(ind_mcc)])
  #clear_output()
  table = tabulate(temp_row, headers, tablefmt='simple')
  temp_row.append([" " for i in range(14)])
  rows = rows + temp_row
  print(table)
  print("\n\n")

Classifier    Random-Set      Train-Accuracy    Test-Accuracy    Ind-Accuracy    Train-Specificity    Test-Specificity    Ind-Specificy    Train-Sensitivity    Test-Sensitivity    Ind-Sensitivity    Train-MCC    Test-MCC    Ind-MCC
------------  ------------  ----------------  ---------------  --------------  -------------------  ------------------  ---------------  -------------------  ------------------  -----------------  -----------  ----------  ---------
SVC           Set01                 0.818022         0.687263        0.707634             0.833198            0.702981         0.755725             0.802846            0.671545           0.659542     0.636357    0.374848   0.417218
              Set02                 0.708943         0.584553        0.576336             0.722358            0.59729          0.563359             0.695528            0.571816           0.589313     0.418135    0.169219   0.152827
              Set03                 0.788821         0.679133        0.6

In [11]:
table = tabulate(rows, headers, tablefmt='rst')
print(table)

======================  ============  ==================  ==================  ==================  ===================  ==================  ==================  ===================  ==================  ==================  ===================  ===================  ===================
Classifier              Random-Set    Train-Accuracy      Test-Accuracy       Ind-Accuracy        Train-Specificity    Test-Specificity    Ind-Specificy       Train-Sensitivity    Test-Sensitivity    Ind-Sensitivity     Train-MCC            Test-MCC             Ind-MCC
======================  ============  ==================  ==================  ==================  ===================  ==================  ==================  ===================  ==================  ==================  ===================  ===================  ===================
SVC                     Set01         0.8180216802168022  0.6872628726287263  0.7076335877862594  0.8331978319783196   0.7029810298102982  0.7557251908396947  0.80284